In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3
import gc
from IPython.display import clear_output
from collections import defaultdict
import os

from functools import lru_cache

def sum_2_tuples(a, b):
    return tuple(map(sum, zip(a, b)))

File paths:

In [2]:
TRADES_DIR = '../TRADES/USD_TOM_trades_Micex_02_10.feather'
LOB_DIR = '../Data_lob/'

SIGNATURE_FILE = './signature.db3'

TOXIC_BUY_MOMENTS_PATH = '../0_CommonFiles/toxic/toxic_buy_moments_1m_02_06.feather'
TOXIC_SELL_MOMENTS_PATH = '../0_CommonFiles/toxic/toxic_sell_moments_1m_02_06.feather'

TOXIC_SELL_SIGNATURE_PATH = '../0_CommonFiles/toxic/toxic_sign_sell.npy'
TOXIC_BUY_SIGNATURE_PATH = '../0_CommonFiles/toxic/toxic_sign.npy'

Experiment details:

In [3]:
# Signature hedger
volume = 10_000 # in contracts

look_back_time = np.timedelta64(30, 's')

hedge_params_df = pd.DataFrame({
    'zone_number': [1, 2, 3],
    'distance': [1, 5, 10],
    'proportion': [0.3, 0.2, 0.1]
})

# 1. Generate 1m trades of toxic client

In [4]:
trades = pd.read_feather(TRADES_DIR)
trades = trades[0:0]
trades

,Time,PRICE,SIZE,AGGRESSOR_SIDE,TRADE_VALUE


In [5]:
toxic_buy_moments = pd.read_feather(TOXIC_BUY_MOMENTS_PATH)

In [6]:
toxic_buy_moments_2 = pd.DataFrame(index=toxic_buy_moments['Time']).between_time('10:00', '23:50')

In [7]:
toxic_buy_moments_2

""
Time
2021-02-01 10:01:36.771
2021-02-01 10:05:02.883
2021-02-01 10:05:11.061
2021-02-01 10:05:23.526
2021-02-01 10:07:27.929
...
2021-06-30 19:17:20.554
2021-06-30 19:30:16.189
2021-06-30 19:31:46.414


In [8]:
toxic_buy_moments_2_index = toxic_buy_moments_2.index

In [9]:
# https://stackoverflow.com/questions/43408621/add-a-row-at-top-in-pandas-dataframe
for mom in toxic_buy_moments_2_index:
    trades.loc[-1, ('Time', 'AGGRESSOR_SIDE', 'PRICE', 'SIZE', 'TRADE_VALUE')] = [mom, 'B', -1, 1000, -1000]
    trades.index = trades.index - 1

In [10]:
trades

,Time,PRICE,SIZE,AGGRESSOR_SIDE,TRADE_VALUE
-15683,2021-02-01 10:01:36.771,-1.0,1000.0,B,-1000.0
-15682,2021-02-01 10:05:02.883,-1.0,1000.0,B,-1000.0
-15681,2021-02-01 10:05:11.061,-1.0,1000.0,B,-1000.0
-15680,2021-02-01 10:05:23.526,-1.0,1000.0,B,-1000.0
-15679,2021-02-01 10:07:27.929,-1.0,1000.0,B,-1000.0
...,...,...,...,...,...
-6,2021-06-30 19:17:20.554,-1.0,1000.0,B,-1000.0
-5,2021-06-30 19:30:16.189,-1.0,1000.0,B,-1000.0
-4,2021-06-30 19:31:46.414,-1.0,1000.0,B,-1000.0
-3,2021-06-30 19:41:04.630,-1.0,1000.0,B,-1000.0


In [11]:
toxic_sell_moments = pd.read_feather(TOXIC_SELL_MOMENTS_PATH)

In [12]:
toxic_sell_moments_2 = pd.DataFrame(index=toxic_sell_moments['Time']).between_time('10:00', '23:50')

In [13]:
toxic_sell_moments_2

""
Time
2021-02-01 10:00:09.015
2021-02-01 10:04:36.332
2021-02-01 10:04:57.946
2021-02-01 10:05:19.015
2021-02-01 10:08:27.788
...
2021-06-30 19:03:41.640
2021-06-30 19:05:10.360
2021-06-30 19:05:24.713


In [14]:
toxic_sell_moments_2_index = toxic_sell_moments_2.index

In [15]:
# https://stackoverflow.com/questions/43408621/add-a-row-at-top-in-pandas-dataframe
for mom in toxic_sell_moments_2_index:
    trades.loc[-1, ('Time', 'AGGRESSOR_SIDE', 'PRICE', 'SIZE', 'TRADE_VALUE')] = [mom, 'S', -1, 1000, -1000]
    trades.index = trades.index - 1

In [16]:
trades.sort_values(by = 'Time', inplace=True)
trades.reset_index(drop = True, inplace=True)

In [17]:
trades

,Time,PRICE,SIZE,AGGRESSOR_SIDE,TRADE_VALUE
0,2021-02-01 10:00:09.015,-1.0,1000.0,S,-1000.0
1,2021-02-01 10:01:36.771,-1.0,1000.0,B,-1000.0
2,2021-02-01 10:04:36.332,-1.0,1000.0,S,-1000.0
3,2021-02-01 10:04:57.946,-1.0,1000.0,S,-1000.0
4,2021-02-01 10:05:02.883,-1.0,1000.0,B,-1000.0
...,...,...,...,...,...
31125,2021-06-30 19:30:16.189,-1.0,1000.0,B,-1000.0
31126,2021-06-30 19:31:46.414,-1.0,1000.0,B,-1000.0
31127,2021-06-30 19:40:36.010,-1.0,1000.0,S,-1000.0
31128,2021-06-30 19:41:04.630,-1.0,1000.0,B,-1000.0


# 2. Prepare signature (t0 + 1sec, t0 + 2 sec, ..., t0 + 30 sec)

In [40]:
arr_b = np.load(TOXIC_BUY_SIGNATURE_PATH)

In [41]:
arr_b

array([0.        , 0.00060652, 0.00076583, 0.00084845, 0.00092663,
       0.00097044, 0.00101048, 0.00105958, 0.00112067, 0.00118184,
       0.00122941, 0.00128676, 0.00131879, 0.00137308, 0.00138769,
       0.00141246, 0.00144499, 0.00147672, 0.00151539, 0.00154786,
       0.00155572, 0.00156564, 0.00157389, 0.00160786, 0.00162569,
       0.00163014, 0.00164679, 0.00167861, 0.00171992, 0.0017324 ,
       0.00174107, 0.00175855, 0.00178216, 0.00181039, 0.00186252,
       0.00188851, 0.00191741, 0.00190683, 0.00193289, 0.00198366,
       0.00200462, 0.00202701, 0.00205765, 0.00209319, 0.00213083,
       0.00215138, 0.00216443, 0.00218651, 0.00221448, 0.00221694,
       0.00222681, 0.00225959, 0.00225123, 0.00227095, 0.00229199,
       0.00229888, 0.00232601, 0.00232881, 0.00234191, 0.00235477,
       0.00240601, 0.0024529 , 0.00246497, 0.00248385, 0.00249143,
       0.00251261, 0.0025324 , 0.00254368, 0.00254669, 0.00253681,
       0.00257654, 0.00262445, 0.00263123, 0.00263597, 0.00262

In [43]:
arr_b = arr_b[::3]

In [44]:
arr_b = arr_b[1:31]

In [45]:
arr_b

array([0.00118184, 0.00151539, 0.00167861, 0.00191741, 0.00215138,
       0.00229199, 0.00248385, 0.00263123, 0.00277063, 0.00285221,
       0.00292325, 0.00303545, 0.00311994, 0.00316936, 0.00319025,
       0.00328976, 0.00342867, 0.00351475, 0.00354547, 0.00355494,
       0.00356097, 0.00366147, 0.0037261 , 0.00379613, 0.00385422,
       0.00397341, 0.00398439, 0.0039669 , 0.00403308])

In [46]:
arr_s = np.load(TOXIC_SELL_SIGNATURE_PATH)
arr_s = arr_s[::3]
arr_s = arr_s[1:31]
arr_s

array([-0.00103321, -0.00126052, -0.00144338, -0.00159671, -0.00169068,
       -0.00183257, -0.0020072 , -0.00211205, -0.00225794, -0.00236021,
       -0.00244329, -0.0025295 , -0.00264395, -0.00282395, -0.0029191 ,
       -0.00298147, -0.00300586, -0.00300877, -0.00307504, -0.00312307,
       -0.00319003, -0.00323405, -0.00331353, -0.00332765, -0.00339945,
       -0.00338945, -0.00344584, -0.00354666, -0.00357928, -0.00356825])

In [51]:
print({
    'B': {(1000, 1001): arr_b},
    'S': {(1000, 1001): arr_s}
    })

{'B': {(1000, 1001): array([0.00118184, 0.00151539, 0.00167861, 0.00191741, 0.00215138,
       0.00229199, 0.00248385, 0.00263123, 0.00277063, 0.00285221,
       0.00292325, 0.00303545, 0.00311994, 0.00316936, 0.00319025,
       0.00328976, 0.00342867, 0.00351475, 0.00354547, 0.00355494,
       0.00356097, 0.00366147, 0.0037261 , 0.00379613, 0.00385422,
       0.00397341, 0.00398439, 0.0039669 , 0.00403308])}, 'S': {(1000, 1001): array([-0.00103321, -0.00126052, -0.00144338, -0.00159671, -0.00169068,
       -0.00183257, -0.0020072 , -0.00211205, -0.00225794, -0.00236021,
       -0.00244329, -0.0025295 , -0.00264395, -0.00282395, -0.0029191 ,
       -0.00298147, -0.00300586, -0.00300877, -0.00307504, -0.00312307,
       -0.00319003, -0.00323405, -0.00331353, -0.00332765, -0.00339945,
       -0.00338945, -0.00344584, -0.00354666, -0.00357928, -0.00356825])}}


# 3. Build signature functions

In [18]:
signature_table = {
    'B': {
        # toxic signature
        (1000, 1001): [0.00084845, 0.00101048, 0.00118184, 0.00131879, 0.00141246,
       0.00151539, 0.00156564, 0.00162569, 0.00167861, 0.00174107,
       0.00181039, 0.00191741, 0.00198366, 0.00205765, 0.00215138,
       0.00221448, 0.00225959, 0.00229199, 0.00232881, 0.00240601,
       0.00248385, 0.0025324 , 0.00253681, 0.00263123, 0.00262157,
       0.00270485, 0.00277063, 0.00278913, 0.00280792, 0.00285221],
    },
    'S': {
        # toxic signature
        (1000, 1001): [-0.00103321, -0.00126052, -0.00144338, -0.00159671, -0.00169068,
       -0.00183257, -0.0020072 , -0.00211205, -0.00225794, -0.00236021,
       -0.00244329, -0.0025295 , -0.00264395, -0.00282395, -0.0029191 ,
       -0.00298147, -0.00300586, -0.00300877, -0.00307504, -0.00312307,
       -0.00319003, -0.00323405, -0.00331353, -0.00332765, -0.00339945,
       -0.00338945, -0.00344584, -0.00354666, -0.00357928, -0.00356825]
    }
}


def build_delta_signature(signature_table):
    output = {}
    for side in signature_table:
        if side not in output:
            output[side] = {}
        
        for size, array in signature_table[side].items():
            # for 1..N we store only delta - change from previous moment
            delta_signature = np.array(array[1:]) - np.array(array[:-1])
            
            '''
            for initial time T0 we store array[0],
            price jump before deal and after deal
            '''
            delta_signature = np.insert(delta_signature, obj=0, values=array[0]) 
            
            output[side][size] = delta_signature
            
    return output

delta_signature_table = build_delta_signature(signature_table)

In [19]:
def build_multitrades_signature(trades_before):
    signature = defaultdict(float)

    for trade in trades_before.itertuples():
        side = trade.AGGRESSOR_SIDE
        for size_range in signature_table[side]:
            if size_range[0] <= trade.SIZE <= size_range[1]:
                break

        delta_signature = delta_signature_table[side][size_range]

        times = {trade.Time + np.timedelta64(sec, 's') : delta_signature[sec] 
               for sec in range(30)}

        for k, v in times.items():
            signature[k] += v
        
    signature = (sorted(signature.items()))

    reconstructed_signature = []
    total = 0

    for (k,v) in signature:
        total += v
        reconstructed_signature.append((k, total))

    return reconstructed_signature

# 3. Hedge functions

In [20]:
def get_trades_before(moment):
    global look_back_time, trades
    moment_before = moment - look_back_time
    index1 = trades.Time.searchsorted(moment_before, side='left')
    index2 = trades.Time.searchsorted(moment, side='right')
    trades_before = trades.iloc[index1 : index2]
    return trades_before

In [21]:
'''
@returns 
zone1time - time, when zone1 ends and zone2 starts
zone2time - time, when zone2 ends and zone3 starts
'''
def get_zone_times(moment, side, skip_empty_trades = False):
    #1 get trades before our trade
    trades_before = get_trades_before(moment)
    
    if len(trades_before) == 0:
        if skip_empty_trades == False:
            #Columns: [Time, AGGRESSOR_SIDE, PRICE, SIZE, TRADE_VALUE]
            trades_before.loc[-1] = [moment, side, -1.0, 1000, -1000.0]  # adding a row
            trades_before.index = trades_before.index + 1  # shifting index
            trades_before = trades_before.sort_index()  # sorting by index
        else:
            return None, None

    #2 mix them into signature
    reference_signature = build_multitrades_signature(trades_before)
    #3 find three zones
    mmax = max(reference_signature, key = lambda item: item[1])
    mmin = min(reference_signature, key = lambda item: item[1])
    
    if side == 'B':
        zone1price = mmin[1] + (mmax[1] - mmin[1]) * 0.33
        zone2price = mmin[1] + (mmax[1] - mmin[1]) * 0.66
    
        for (k, v) in reference_signature:
            if v >= zone1price:
                #print(k)
                zone1time = k
                break
            
        for (k, v) in reference_signature:
            if v >= zone2price:
                #print(k)
                zone2time = k
                break
    else:
        zone1price = mmax[1] - (mmax[1] - mmin[1]) * 0.33
        zone2price = mmax[1] - (mmax[1] - mmin[1]) * 0.66
        
        for (k, v) in reference_signature:
            if v <= zone1price:
                #print(k)
                zone1time = k
                break
            
        for (k, v) in reference_signature:
            if v <= zone2price:
                #print(k)
                zone2time = k
                break
                
    #4 return result
    return zone1time, zone2time

In [22]:
def make_hedge(lob, datetime, size, side):
    #lob = lob[lob['Time'] >= datetime]
    time_index = lob['Time'].searchsorted(datetime, side='right')
        
    if time_index >= len(lob):
        return (0, 0.0)
    row = lob.iloc[time_index] # first timestamp >= datetime
    
    quote_side = 'BID' if (side == 'S') else 'ASK'
    
    maxsize = sum([row[quote_side + '_SIZE' + str(level)] for level in range(1, 11)])
        
    if maxsize < size:
        raise ValueError(f'size {size} is greater than available {maxsize}')
    
    size_hedged, sum_hedged = 0, 0.0
    level = 1
    while size_hedged < size and level < 11:
        portion = min(size - size_hedged, row[quote_side + '_SIZE' + str(level)])
        price = row[quote_side + '_PRICE' + str(level)]
        size_hedged += portion
        sum_hedged += portion * price
        print(f'added {portion} by {price}')
        level += 1
        
    print('HEDGE ended')
    return (size_hedged, sum_hedged)

In [23]:
class LOBCache:
    def __init__(self):
        pass
    
    def get_lob(self, datetime):
        #print(datetime)
        filename = datetime.strftime('LOB_%m%d.feather')
        return self.__get_lob_by_filename(filename)
        pass
    
    @lru_cache(maxsize=1)
    def __get_lob_by_filename(self, filename):
        print(filename)
        lob = pd.read_feather(LOB_DIR + filename).dropna()
        lob['BID_SIZE10'] = 99_999_999
        lob['ASK_SIZE10'] = 99_999_999
        return lob

In [24]:
cache = LOBCache()

In [25]:
def calc_hedge_signature(cache, moment, hedge_params_df):
    curr_moment = moment
    print(hedge_params_df)

    lob = cache.get_lob(curr_moment)
    remain = volume
    total = None
    
    zone1time, zone2time = get_zone_times(curr_moment, side, skip_empty_trades = True)
    print('zone1time, zone2time','\n',zone1time,'\n', zone2time)
    if zone1time is None:
        remain = 0
        total = (-2, -2.0)
    while remain > 0:
        print(curr_moment)
        
        # select our zone
        if moment <= curr_moment <= zone1time:
            print('we are in 1st zone')
            zone = 1
        elif zone1time < curr_moment <= zone2time:
            print('we are in 2nd zone')
            zone = 2
        else:
            print('we are in 3rd zone')
            zone = 3
        
        row = hedge_params_df[hedge_params_df['zone_number'] == zone]
        size_contracts = int(volume * row['proportion'])
        size_contracts = min(size_contracts, remain)

        hedge_result = make_hedge(lob, curr_moment, size = size_contracts, side = side)

        interval =  row['distance'].iloc[0]
        curr_moment = curr_moment + np.timedelta64(interval, 's')
        if hedge_result is None:
            break
        
        if hedge_result[0] == 0:
            return (-1, -1.0)
        
        if total is None:
            total = hedge_result
        else:
            total = sum_2_tuples(total, hedge_result)
            
        print(hedge_result)
        remain -= hedge_result[0]
        if remain == 0:
            break
            
    del lob
            
    if remain > 0:
        return None
    else:
        return total

# 5. Calculate and save results in DB

In [26]:
#CREATE TABLE RESULT(MOMENT TEXT, SIDE TEXT, QUANTITY INTEGER, SUM REAL, PRIMARY KEY(MOMENT,SIDE));
sign_sql = sqlite3.connect(SIGNATURE_FILE)
sign_sql.execute('CREATE TABLE IF NOT EXISTS RESULT(MOMENT TEXT, SIDE TEXT, QUANTITY INTEGER, SUM REAL, PRIMARY KEY(MOMENT,SIDE));')
cur=sign_sql.cursor()

In [27]:
records = []

for side in ('B', 'S'):
    if side == 'B':
        moments = toxic_buy_moments_2_index
    elif side == 'S':
        moments = toxic_sell_moments_2_index

    for moment in moments:
        record = (str(moment),side)
        query='select exists(select 1 from result where moment=? and side=? collate nocase) limit 1'
        # 'query' RETURNS 1 IF USERNAME EXISTS OR 0 IF NOT, AS INTEGER(MAYBE). 'collate nocase'= CASE INSENSITIVE, IT'S OPTIONAL
        check=cur.execute(query,record) 
        if check.fetchone()[0]==1:
            print(f'Moment already available ' + str(record))
            continue

        print('calculating for moment', moment)
        result = calc_hedge_signature(cache, moment, hedge_params_df)

        query='INSERT OR REPLACE INTO result(moment, side, quantity, sum) VALUES(?, ?, ?, ?);'
        if result:
            record = (str(moment), side, int(result[0]), result[1])
        else:
            record = (str(moment), side, -1, -1)

        print(record)
        cur.execute(query, record)
        records.append(record)

        gc.collect()

        if len(records) % 100 == 0:
            clear_output(wait=False)

calculating for moment 2021-06-30 15:00:59.309000
   zone_number  distance  proportion
0            1         1         0.3
1            2         5         0.2
2            3        10         0.1
zone1time, zone2time 
 2021-06-30 15:00:51.524000 
 2021-06-30 15:01:04.309000
2021-06-30 15:00:59.309000
we are in 2nd zone
added 553 by 72.93
added 612 by 72.9275
added 520 by 72.925
added 315 by 72.9225
HEDGE ended
(2000, 145853.5075)
2021-06-30 15:01:04.309000
we are in 2nd zone
added 94 by 72.9325
added 903 by 72.93
added 512 by 72.9275
added 491 by 72.925
HEDGE ended
(2000, 145856.5)
2021-06-30 15:01:09.309000
we are in 3rd zone
added 738 by 72.93
added 262 by 72.9275
HEDGE ended
(1000, 72929.345)
2021-06-30 15:01:19.309000
we are in 3rd zone
added 44 by 72.9325
added 741 by 72.93
added 215 by 72.9275
HEDGE ended
(1000, 72929.57250000001)
2021-06-30 15:01:29.309000
we are in 3rd zone
added 500 by 72.9325
added 212 by 72.93
added 288 by 72.9275
HEDGE ended
(1000, 72930.53)
2021-06-30 15

HEDGE ended
(3000, 218915.36250000002)
2021-06-30 15:32:17.059000
we are in 1st zone
added 160 by 72.98
added 350 by 72.9775
added 609 by 72.975
added 400 by 72.9725
added 673 by 72.97
added 503 by 72.9675
added 305 by 72.965
HEDGE ended
(3000, 218915.48750000002)
2021-06-30 15:32:18.059000
we are in 1st zone
added 160 by 72.98
added 450 by 72.9775
added 459 by 72.975
added 400 by 72.9725
added 873 by 72.97
added 403 by 72.9675
added 255 by 72.965
HEDGE ended
(3000, 218915.98750000002)
2021-06-30 15:32:19.059000
we are in 1st zone
added 260 by 72.98
added 515 by 72.9775
added 225 by 72.975
HEDGE ended
(1000, 72977.58750000001)
('2021-06-30 15:32:16.059000', 'S', 10000, 729724.425)
calculating for moment 2021-06-30 15:32:23.507000
   zone_number  distance  proportion
0            1         1         0.3
1            2         5         0.2
2            3        10         0.1
zone1time, zone2time 
 2021-06-30 15:32:24.059000 
 2021-06-30 15:32:30.507000
2021-06-30 15:32:23.507000
we are

added 162 by 72.86
added 350 by 72.8575
added 300 by 72.855
added 188 by 72.8525
HEDGE ended
(1000, 72856.215)
('2021-06-30 16:05:05.211000', 'S', 10000, 728516.955)
calculating for moment 2021-06-30 16:09:44.515000
   zone_number  distance  proportion
0            1         1         0.3
1            2         5         0.2
2            3        10         0.1
zone1time, zone2time 
 2021-06-30 16:09:50.515000 
 2021-06-30 16:09:57.515000
2021-06-30 16:09:44.515000
we are in 1st zone
added 350 by 72.9075
added 351 by 72.905
added 2000 by 72.9025
added 299 by 72.9
HEDGE ended
(3000, 218709.38)
2021-06-30 16:09:45.515000
we are in 1st zone
added 300 by 72.9075
added 351 by 72.905
added 2000 by 72.9025
added 349 by 72.9
HEDGE ended
(3000, 218709.005)
2021-06-30 16:09:46.515000
we are in 1st zone
added 250 by 72.9075
added 451 by 72.905
added 2000 by 72.9025
added 299 by 72.9
HEDGE ended
(3000, 218709.13)
2021-06-30 16:09:47.515000
we are in 1st zone
added 200 by 72.9075
added 501 by 72.90

added 638 by 72.895
added 216 by 72.8925
HEDGE ended
(3000, 218696.37250000003)
2021-06-30 17:40:45.290000
we are in 1st zone
added 300 by 72.905
added 450 by 72.9025
added 250 by 72.9
HEDGE ended
(1000, 72902.625)
('2021-06-30 17:40:42.290000', 'S', 10000, 728977.0625)
calculating for moment 2021-06-30 17:45:19.116000
   zone_number  distance  proportion
0            1         1         0.3
1            2         5         0.2
2            3        10         0.1
zone1time, zone2time 
 2021-06-30 17:45:25.116000 
 2021-06-30 17:45:32.116000
2021-06-30 17:45:19.116000
we are in 1st zone
added 100 by 72.9425
added 450 by 72.94
added 350 by 72.9375
added 250 by 72.935
added 550 by 72.9325
added 950 by 72.93
added 300 by 72.9275
added 50 by 72.925
HEDGE ended
(3000, 218800.0)
2021-06-30 17:45:20.116000
we are in 1st zone
added 150 by 72.9425
added 400 by 72.94
added 350 by 72.9375
added 250 by 72.935
added 650 by 72.9325
added 850 by 72.93
added 300 by 72.9275
added 50 by 72.925
HEDGE end

 2021-06-30 18:02:17.484000
2021-06-30 18:02:16.484000
we are in 2nd zone
added 200 by 73.0375
added 850 by 73.035
added 750 by 73.0325
added 200 by 73.03
HEDGE ended
(2000, 146067.625)
2021-06-30 18:02:21.484000
we are in 3rd zone
added 150 by 73.03
added 298 by 73.0275
added 250 by 73.025
added 250 by 73.0225
added 52 by 73.02
HEDGE ended
(1000, 73025.61)
2021-06-30 18:02:31.484000
we are in 3rd zone
added 15 by 73.015
added 350 by 73.0125
added 635 by 73.01
HEDGE ended
(1000, 73010.95000000001)
2021-06-30 18:02:41.484000
we are in 3rd zone
added 553 by 73.01
added 100 by 73.0075
added 347 by 73.005
HEDGE ended
(1000, 73008.015)
2021-06-30 18:02:51.484000
we are in 3rd zone
added 250 by 72.985
added 374 by 72.9825
added 235 by 72.98
added 141 by 72.9775
HEDGE ended
(1000, 72981.8325)
2021-06-30 18:03:01.484000
we are in 3rd zone
added 172 by 72.985
added 150 by 72.9825
added 85 by 72.98
added 100 by 72.9775
added 350 by 72.975
added 143 by 72.9725
HEDGE ended
(1000, 72978.1625)
2021-

In [28]:
sign_sql.commit()

In [29]:
cur.close()
sign_sql.close()